In [194]:
# 9/20
import numpy as np 
import pandas as pd 
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle


In [3]:
credits = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')
movie = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')

In [4]:
movies = pd.merge(movie,credits,how='inner')

In [5]:
df = movies[['movie_id','title','genres','keywords','overview','cast','crew']]

In [6]:
df = df.dropna()

In [7]:
def fetch_name(obj):
    name = []
    if type(obj) == str:
        obj = json.loads(obj)
    for i in obj:
        name.append(i['name'].replace(" ","").lower())
    return name
def fetch_name_3(obj):
    name = []
    if type(obj) == str:
        obj = json.loads(obj)
    for i in obj[0:3]:
        name.append(i['name'].replace(" ","").lower())
    return name
def fetch_director(obj):
    name = []
    if type(obj) == str:
        obj = json.loads(obj)
    for i in obj:
        if i['job'] == "Director":
            name.append(i['name'].replace(" ","").lower())
    return name

In [49]:
new_df = df[['movie_id','title','overview']]
new_df['genres'] = df['genres'].apply(fetch_name)
new_df['keywords'] = df['keywords'].apply(fetch_name)
new_df['cast'] = df['cast'].apply(fetch_name_3)
new_df['director'] = df['crew'].apply(fetch_director)
new_df['overview'] = df['overview'].apply(lambda x:x.split())

/tmp/ipykernel_42/3357899780.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['genres'] = df['genres'].apply(fetch_name)
/tmp/ipykernel_42/3357899780.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['keywords'] = df['keywords'].apply(fetch_name)
/tmp/ipykernel_42/3357899780.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

In [57]:
new_df['tags'] = new_df['overview'] + new_df['genres'] + new_df['keywords'] +new_df['cast'] + new_df['director'] 

In [51]:
new_df['tags'] = new_df['tags'].apply(lambda x : " ".join(x))

In [52]:
final_df = new_df[['movie_id','title','tags']]

In [59]:
final_df['tags'] = final_df['tags'].apply(lambda x:)

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."
...,...,...,...
4804,9367,El Mariachi,El Mariachi just wants to play his guitar and ...
4805,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...
4806,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic..."
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...


In [62]:
cv = CountVectorizer(max_features=5000, stop_words='english')

In [82]:
vectors = cv.fit_transform(final_df['tags']).toarray()

In [90]:
similarity = cosine_similarity(vectors)

In [198]:
with open('similarity.pkl','wb') as f:
    pickle.dump(similarity,f)

In [189]:
def get_recommedation(movie):
    movie_list = []
    id = final_df[final_df['title'] == movie].index[0]
    movie_list_id = sorted(list(enumerate(similarity[id])),reverse=True,key=lambda x:x[1])
    for i in movie_list_id[1:6]:
        movie_list.append(final_df.loc[i[0]]['title'])
    return movie_list
    
    

In [190]:
get_recommedation('Titan A.E.')


['Spirit: Stallion of the Cimarron',
 'Escape from Planet Earth',
 'Home',
 'This Is England',
 'The 5th Wave']

In [122]:
movie_list = sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])

In [126]:
for i in movie_list[1:6]:
    print(i[0])

539
1194
260
1216
507


In [151]:
final_df[final_df['title'] == "Titan A.E."].index[0]

539

In [192]:
final_df = final_df.drop('tags',axis=1)

In [197]:
with open('data.pkl','wb') as f:
    pickle.dump(final_df, f)